<a href="https://colab.research.google.com/github/ygyashgoyal/VAMP-Sys/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [ ]:
!pip install -q google-generative-ai


ERROR: Could not find a version that satisfies the requirement google-generative-ai (from versions: none)
ERROR: No matching distribution found for google-generative-ai


In [ ]:
import google.generativeai as genai

from google.colab import userdata
API_KEY = "" # insert your GOOGLE API key

# Provide your API key here
genai.configure(api_key=API_KEY)

# Initialize the Gemini Flash Model
model = genai.GenerativeModel('gemini-2.0-flash')


In [ ]:
from PIL import Image

image = Image.open('img7.png')


In [ ]:
prompt = "Analyze this network diagram and list the network only, e.g. Q + W -> R, P.S. the reaction'x' are operators, if there are multiple reactions give them with comma separations, like A -> B, B -> C, etc."

response = model.generate_content([prompt, image])

# print("Model's response:")
print(response.text)


S + E1 -> SE1, SE1 -> E1 + Sp, E2 + Sp -> SpE2, SpE2 -> E2 + Sp, S -> SSp, SSp -> 2Sp


In [ ]:
# ✅ Gradio App with Support for Multi-Reactant Networks (e.g. A + B -> AB)

from transformers import pipeline
import gradio as gr
import networkx as nx
import sympy as sp
from collections import defaultdict
import re

# --- Parsing Functions ---
def parse_species(expr):
    # e.g., "A + B" -> ["A", "B"]
    return [s.strip() for s in re.split(r'\s*[\+\-]\s*', expr)]

def parse_network(input_string):
    edges = []
    reversible_edges = []

    for part in input_string.split(','):
        part = part.strip()
        if '<->' in part:
            lhs, rhs = part.split('<->')
            lhs_species = parse_species(lhs)
            rhs_species = parse_species(rhs)
            reversible_edges.append((lhs_species, rhs_species))
        elif '->' in part:
            lhs, rhs = part.split('->')
            lhs_species = parse_species(lhs)
            rhs_species = parse_species(rhs)
            edges.append((lhs_species, rhs_species))

    return edges, reversible_edges

def build_graph(edges, reversible_edges):
    G = nx.DiGraph()
    for a, b in edges:
        lhs = " + ".join(a)
        rhs = " + ".join(b)
        G.add_edge(lhs, rhs)
    for a, b in reversible_edges:
        lhs = " + ".join(a)
        rhs = " + ".join(b)
        G.add_edge(lhs, rhs)
        G.add_edge(rhs, lhs)
    return G

def analyze_graph(G):
    return {
        "nodes": list(G.nodes),
        "edges": list(G.edges),
        "num_nodes": G.number_of_nodes(),
        "num_edges": G.number_of_edges(),
        "is_cyclic": not nx.is_directed_acyclic_graph(G)
    }

# --- ODE Generator for Complex Reactions ---
def mass_action_odes(edges, reversible_edges):
    species = set()
    odes = defaultdict(lambda: 0)
    rate_counter = 1

    def term(species_list):
        term_expr = 1
        for s in species_list:
            sym = sp.symbols(s)
            species.add(sym)
            term_expr *= sym
        return term_expr

    for lhs_species, rhs_species in edges:
        k = sp.symbols(f'k{rate_counter}')
        rate_counter += 1

        flux = k * term(lhs_species)
        for s in lhs_species:
            sym = sp.symbols(s)
            odes[sym] -= flux
        for s in rhs_species:
            sym = sp.symbols(s)
            odes[sym] += flux

    for lhs_species, rhs_species in reversible_edges:

        kf = sp.symbols(f'k{rate_counter}')
        rate_counter += 1
        kr = sp.symbols(f'k{rate_counter}')
        rate_counter += 1

        forward_flux = kf * term(lhs_species)
        reverse_flux = kr * term(rhs_species)

        for s in lhs_species:
            sym = sp.symbols(s)
            odes[sym] -= forward_flux
            odes[sym] += reverse_flux
        for s in rhs_species:
            sym = sp.symbols(s)
            odes[sym] += forward_flux
            odes[sym] -= reverse_flux

    return dict(odes)

def format_odes(odes):
    return "\n".join([f"d{var}/dt = {sp.simplify(expr)}" for var, expr in odes.items()])

def compute_jacobian(odes):
    variables = list(odes.keys())
    F = sp.Matrix([odes[var] for var in variables])
    J = F.jacobian(variables)
    return sp.pretty(J)

qa = pipeline("text2text-generation", model="google/flan-t5-base")
def process_network(input_string, query):
    edges, reversible_edges = parse_network(input_string)
    G = build_graph(edges, reversible_edges)
    info = analyze_graph(G)

    if 'ode' in query.lower():
        ode_sys = mass_action_odes(edges, reversible_edges)
        return format_odes(ode_sys)

    elif 'jacobian' in query.lower():
        ode_sys = mass_action_odes(edges, reversible_edges)
        return f"Jacobian Matrix:\n{compute_jacobian(ode_sys)}"

    elif 'variables' in query.lower():
        return f"There are {info['num_nodes']} variables: {info['nodes']}"

    elif 'edges' in query.lower():
        return f"Edges: {info['edges']}"

    elif 'cyclic' in query.lower():
        cycles = list(nx.simple_cycles(G))
        if cycles:
            cycles_str = "\n".join([" -> ".join(cycle + [cycle[0]]) for cycle in cycles])
            return f"Cycles found:\n{cycles_str}"
        else:
            return "No cycles found."

    else:
        prompt = f"Given the network with nodes: {info['nodes']} and edges: {info['edges']}, answer the query: {query}"
        answer = qa(prompt, max_length=128)[0]['generated_text']
        return answer

# iface = gr.Interface(
#     fn=process_network,
#     inputs=[
#         gr.Textbox(label="Network Description", placeholder="Example: A->B, B<->C, A + B -> AB"),
#         gr.Textbox(label="Query", placeholder="Ask about variables, ODEs, Jacobian, etc.")
#     ],
#     outputs="text",
#     title="Biological Network Analyzer",
#     description="Now supports multi-reactant reactions like 'A + B -> AB'"
# )

# iface.launch()

print(response.text)
print(process_network(response.text, "give the ode of the network"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


A + B -> C
dA/dt = -A*B*k1
dB/dt = -A*B*k1
dC/dt = A*B*k1
